In [1]:
!pip install arff

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for arff: filename=arff-0.9-py3-none-any.whl size=4949 sha256=f5eaca16b9f3c8bcb1943b3433394701026f010ba5cdf6a06759278c8263fd16
  Stored in directory: c:\users\cri98\appdata\local\pip\cache\wheels\af\3f\c2\9a1a26fd45d8658782cfb94369e2656feef3d2006bc5163142
Successfully built arff


In [1]:
import pandas as pd
import numpy as np
import arff
from tqdm.auto import tqdm
from glob import glob
import os

In [2]:
datasets = [y for x in os.walk("../.") for y in glob(os.path.join(x[0], '*.zip'))]

len(datasets)

32

In [9]:
for dataset in tqdm(datasets):
    df = pd.read_csv(dataset)

    if "_y." in dataset:
        df[[df.columns[-1]]] = df[[df.columns[-1]]].astype(str)
        df[[df.columns[-1]]].to_csv(dataset[3:].replace(".zip", ".csv"))

    classes = "\r\n@attribute class {"
    for value in df[df.columns[-1]].unique():
        classes+= f"'{value}',"

    #df.drop(columns=df.columns[-1], inplace=True)

    arff.dump(dataset[3:].replace(".zip", ".arff"), df.values, relation="none", names=df.columns)

    file = open(dataset[3:].replace(".zip", ".arff"), 'a')

    file.write(classes[:-1]+"}")

  0%|          | 0/32 [00:00<?, ?it/s]

In [7]:
datasets[0][4:]

'\\real\\adult_y.zip'